# <center> INTRODUCTION À PYTHON POUR L'ÉCONOMIE APPLIQUÉE</center>
## <center> DEVOIR 1 : À RENDRE LE LUNDI 26.12.2022</center>
### <center> (AIDE POUR UN CHOIX DE QUESTIONS) </center>
#### <center>Michal Urdanivia (UGA)</center>
#### <center> michal.wong-urdanivia@univ-grenoble-alpes.fr </center>

Les fonctions suivantes considèrent le cas de l'estimation par MCO et utilisent numpy et le calcul sur arrays. Vous pouvez vous en servir en essayant de les comprendre et vous reportant à vos cours d'économétrie pour des détails sur les formules(remarque: mes commentaires sont en anglais)

In [1]:
import numpy as np 


In [2]:
# Fonction pour calculer l'estimateur 
# Remarques: 
# Les arguments sont de series pandas.
# La fonction produit un liste d'éléments dont les coefficients: élément d'indice 0

def ols_beta_hat(Y, X, s_wgt=None, nocons=False):

    n  = len(Y)  
    if X.ndim == 1:
        K = 1
        ind_var = X.name
        X = X.values.reshape((n,1))
    else:
        K = X.shape[1]
        ind_var = list(X.columns)  
        X = X.values 

    dep_var = Y.name
    Y = Y.values.reshape((n,1))

    if s_wgt is None:
       sw = 1 
       s_wgt_var = 'None'
    else:
       s_wgt_var = s_wgt.name                             
       sw = np.asarray(s_wgt/s_wgt.mean()).reshape((-1,1))

    if not nocons:
        X = np.concatenate((np.ones((n,1)), X), axis=1) 

    if K ==1 :
        ind_var = ['constant', ind_var]
    else : 
        ind_var = ['constant'] + ind_var   
        K += 1

    XX  = (sw * X).T @ X
    XY  = (sw * X).T @ Y

    beta_hat = np.linalg.solve(XX, XY)
    ehat    = X @ beta_hat
    score_i   = sw * X * (Y - ehat)
    return[beta_hat, ehat, XX, XY, X, Y, n, K, ind_var, dep_var, sw, score_i, s_wgt_var]




**Application sur données**: le code suivant télécharge les données utilisées qui correspondent à un article de 1995 [""Using Geographic Variation in College Proximity to Estimate the Return to Schooling."](https://davidcard.berkeley.edu/papers/geo_var_schooling.pdf).  

In [3]:
import pandas as pd
import statsmodels.api as sm

In [4]:
card1995_df = pd.read_stata("https://www.ssc.wisc.edu/~bhansen/econometrics/Card1995.dta")
card1995_df.info()   # Affichage d'informations.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3613 entries, 0 to 3612
Data columns (total 52 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        3613 non-null   int16  
 1   nearc2    3613 non-null   int8   
 2   nearc4    3613 non-null   int8   
 3   nearc4a   3613 non-null   int8   
 4   nearc4b   3613 non-null   int8   
 5   ed76      3613 non-null   int8   
 6   ed66      3613 non-null   int8   
 7   age76     3613 non-null   int8   
 8   daded     3613 non-null   float32
 9   nodaded   3613 non-null   int8   
 10  momed     3613 non-null   float32
 11  nomomed   3613 non-null   int8   
 12  weight    3613 non-null   int32  
 13  momdad14  3613 non-null   int8   
 14  sinmom14  3613 non-null   int8   
 15  step14    3613 non-null   int8   
 16  reg661    3613 non-null   int8   
 17  reg662    3613 non-null   int8   
 18  reg663    3613 non-null   int8   
 19  reg664    3613 non-null   int8   
 20  reg665    3613 non-null   int8

In [5]:
card1995_df = card1995_df.dropna(subset=['wage76', 'ed76', 'black', 'smsa76r', 'south66', 'age76', 'nearc4'])
card1995_df.shape

(3017, 52)

In [6]:
card1995_df['exper'] = card1995_df['age76'] - 16-6
card1995_df['expersq100'] = (card1995_df['age76'] - 16-6)**2/100
card1995_df['lwage76'] = np.log(card1995_df.wage76)
X = card1995_df[['ed76', 'exper', 'expersq100', 'black', 'smsa76r', 'south66']]
Y = card1995_df['lwage76']
Z = card1995_df['nearc4'] 

In [7]:
# Application de la fonction
coeffs = ols_beta_hat(Y, X)
coeffs[0]

array([[ 5.51386287],
       [ 0.03447452],
       [ 0.03805097],
       [ 0.04525096],
       [-0.18826047],
       [ 0.17610176],
       [-0.1020169 ]])

In [8]:
# Comparaison avec Statsmodels

In [9]:
import statsmodels.api as sm
model = sm.OLS(Y, sm.add_constant(X))
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                lwage76   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     177.1
Date:                Thu, 24 Nov 2022   Prob (F-statistic):          1.95e-193
Time:                        11:41:39   Log-Likelihood:                -1459.0
No. Observations:                3017   AIC:                             2932.
Df Residuals:                    3010   BIC:                             2974.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.5139      0.043    127.565      0.0

In [10]:
# Fonction pour calculer la variance dans le cas hétéroscédastique sans clustering
# Remarques: 
# Les arguments sont l'output de la fonction pour estimer
def ols_vcov_hc(ols_beta_hat):

    ehat = ols_beta_hat[1]
    XX = ols_beta_hat[2]
    Y = ols_beta_hat[5]
    n = ols_beta_hat[6]
    K = ols_beta_hat[7]
    sw = ols_beta_hat[10]
    score_i=   ols_beta_hat[11]    # n x K m
 
     
     # Calculate heteroscedastic robust variance.
    fsc = n/(n-K)                   # Finite
    omega = fsc*(score_i.T @ score_i) # K X 
     
    iXX = np.linalg.inv(XX)
    vcov_beta_hat = iXX @ omega @ iXX.T


    vcov_type='HC'

    return [vcov_beta_hat, vcov_type]

In [11]:
# Application

vcov = ols_vcov_hc(coeffs)


In [12]:
# Écart-types estimés
std = np.sqrt(np.diag(ols_vcov_hc(coeffs)[0]))
std

array([0.04407436, 0.00282983, 0.0024297 , 0.01455765, 0.01899232,
       0.01592654, 0.01659616])